# Atlântico Academy Future
### VISÃO GERAL
Uma empresa contratante deseja estabelecer termos de maior relevância em um documento específico. Neste caso, considere o histórico de exames, consultas e procedimentos realizados por um paciente. Um sistema deve ser desenvolvido para que o médico possa ter uma visão geral do histórico do paciente sem a necessidade de analisar documento por documento. Com
base nesta importância, vamos desenvolver uma etapa deste sistema. Tokenizar um texto,realizar remoção de stopwords, aplicar o processo de lematização e fazer uma análise quantitativa deste. Neste caso, vamos comparar duas estratégias , se possível. A primeira utilizando a lib stanza e a segunda uma análise com base em acesso a um dicionário.

# 1) Carregar o conjunto de documentos em PDF e armazená-los em alguma estrutura de dados

In [1]:
import PyPDF2
import os
import re

def read_pdf_file(file):
    pdf_file = open(file, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    page_content_all= ''
    for i in range(number_of_pages):
        page_content = read_pdf.getPage(i).extractText()
        page_content_all += page_content
    return page_content_all

In [2]:
def load_pfds(patch_files):
    os.chdir(patch_files)
    file_base_all = []
    for file in os.listdir():
        if file.endswith(".pdf"):
            file_base_all.append(read_pdf_file(os.path.join(patch_files, file)))
    return file_base_all

In [3]:
def join_and_remove_breaks(base):
    list_parsed = []
    for i in range(len(base)):
        list_parsed.append(re.sub('\n', '', base[i]))
    return list_parsed

In [4]:
#corpus = join_and_remove_breaks(load_pfds(r'C:\Users\r211315\Documents\ia_express\arquivos'))
corpus = join_and_remove_breaks(load_pfds(r'/home/vagnersv/my_tensorflow/arquivos'))

In [5]:
len(corpus)

2

In [6]:
corpus

['Uma rápida raposa marrom pula sobre o cão preguiçoso. Que a raposa! ',
 'Uma rápida raposa marrom pula sobre a raposa preguiçoso. QUE RAPASA DO BARALHO! ']

# 2) Realizar o pré-processamento destes ( tokenização e remoção de stop words, deixar todos os caracteres minúsculos...)

Remoção de Palavras Vazias(e.g., artigos, preposições, etc.), que possuem alta frequência em todos os documentos, podem ser removidas da contagem para melhorar a distinção entre documentos

[CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [7]:
for i in range(len(corpus)):
    corpus[i]= corpus[i].lower()

In [8]:
corpus

['uma rápida raposa marrom pula sobre o cão preguiçoso. que a raposa! ',
 'uma rápida raposa marrom pula sobre a raposa preguiçoso. que rapasa do baralho! ']

In [9]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

def removestopwords(texto):
    frases = []
    for palavras in texto:
        semstop = [p for p in palavras.split() if p not in stopwords]
        frases.append(semstop)
    return frases

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vagnersv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
corpus_sem_stopwords = removestopwords(corpus)

In [11]:
def convert_list_to_string(corpus, seperator=' '):
    return  seperator.join(corpus)

In [12]:
corpus_sem_stopwords= list(map(convert_list_to_string, corpus_sem_stopwords))

In [13]:
corpus_sem_stopwords

['rápida raposa marrom pula sobre cão preguiçoso. raposa!',
 'rápida raposa marrom pula sobre raposa preguiçoso. rapasa baralho!']

In [14]:
from nltk.stem import PorterStemmer
import string

#######
# https://stackoverflow.com/questions/26126442/combining-text-stemming-and-removal-of-punctuation-in-nltk-and-scikit-learn
# based on http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(texto):
    tokens = nltk.word_tokenize(texto)
    tokens = [i for i in tokens if i not in string.punctuation]
    stems = stem_tokens(tokens, stemmer)
    return stems

In [15]:
corpus_sem_stopwors_tokenizada= list(map(tokenize, corpus_sem_stopwords))

# 3) Lematização com a Lib stanza

In [16]:
def download_stanza_portugues():
    """
    Faz o download do stanza em portugues
    """
    stanza.download(lang='pt')

In [17]:
# Só é necessário executar o download uma vez
#download_stanza_portugues()

In [18]:
def tokenizer_and_lemmatizer(text):
    """
        Performs tokenization and lemmatization on input text

    Args:
        text: A string with the content of the text

    Returns:
        A stanza Document with the tokens and lemmas

    """
    nlp = stanza.Pipeline('pt', processors='tokenize,mwt,pos,lemma')
    return nlp(text)

In [19]:
def show_nlp_doc(doc):
    """
    Imprime os tokens (somente para debug)
    """
    sentence_id = 0
    for sentence in doc.sentences:
        sentence_id += 1
        print('\nSentença {}:'.format(sentence_id))
        for word in sentence.words:
            print('palavra = {}, lema = {}, id = {}'.format(word.text, word.lemma, word.id))


In [20]:
#directory_path = r'C:\Users\r211315\Documents\ia_express\arquivos'
directory_path = r'/home/vagnersv/my_tensorflow/arquivos'

In [21]:
import stanza

txt3_processed = tokenizer_and_lemmatizer(corpus[0])

2021-12-01 22:28:40 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |

2021-12-01 22:28:41 INFO: Use device: cpu
2021-12-01 22:28:41 INFO: Loading: tokenize
2021-12-01 22:28:41 INFO: Loading: mwt
2021-12-01 22:28:41 INFO: Loading: pos
2021-12-01 22:28:43 INFO: Loading: lemma
2021-12-01 22:28:43 INFO: Done loading processors!


In [22]:
show_nlp_doc(txt3_processed)


Sentença 1:
palavra = uma, lema = um, id = 1
palavra = rápida, lema = rápido, id = 2
palavra = raposa, lema = raposa, id = 3
palavra = marrom, lema = marrom, id = 4
palavra = pula, lema = pular, id = 5
palavra = sobre, lema = sobre, id = 6
palavra = o, lema = o, id = 7
palavra = cão, lema = cão, id = 8
palavra = preguiçoso, lema = preguiçoso, id = 9
palavra = ., lema = ., id = 10

Sentença 2:
palavra = que, lema = que, id = 1
palavra = a, lema = o, id = 2
palavra = raposa, lema = raposa, id = 3
palavra = !, lema = !, id = 4


# 4) Lematização manual com inspiração no trabalho descrito no [link](https://github.com/rikarudo/LemPORT) (Atividade desafio)

# 5) Implementar API para determinar as seguintes informações do resultados obtidos em 3 e/ou 4 :

# 5.1) Term Frequency (TF):
𝑇𝐹 = 𝑞𝑡𝑑 𝑑𝑒 𝑜𝑐𝑜𝑟𝑟ê𝑛𝑐𝑖𝑎 𝑑𝑜 𝑡𝑒𝑟𝑚𝑜 𝑒𝑚 𝑢𝑚 𝑡𝑒𝑥𝑡𝑜 / 𝑞𝑢𝑎𝑛𝑡𝑖𝑑𝑎𝑑𝑒 𝑡𝑜𝑡𝑎𝑙 𝑑𝑒 𝑝𝑎𝑙𝑎𝑣𝑟𝑎𝑠 𝑑𝑜 𝑡𝑒𝑥𝑡𝑜

Referência:
[(Calculate TF-IDF in NLP (Simple Example)](https://youtu.be/vZAXpvHhQow)

[scikit-exemplos](https://dadosaocubo.com/nlp-com-scikit-learn/)

[Turing](https://github.com/turing-usp/BoW-e-TFIDF/blob/master/BoW_e_TFIDF.ipynb)

https://www.computersciencemaster.com.br/como-implementar-o-tf-idf-em-python/

In [23]:
def map_occurrences(string_list):
    string_count = {}
    for item in string_list:
        if item not in string_count:
            count = string_list.count(item)
            string_count[item] = count
    return string_count

In [24]:
ocorrencias_termo= list(map(map_occurrences, corpus_sem_stopwors_tokenizada))

In [25]:
ocorrencias_termo

[{'rápida': 1,
  'raposa': 2,
  'marrom': 1,
  'pula': 1,
  'sobr': 1,
  'cão': 1,
  'preguiçoso': 1},
 {'rápida': 1,
  'raposa': 2,
  'marrom': 1,
  'pula': 1,
  'sobr': 1,
  'preguiçoso': 1,
  'rapasa': 1,
  'baralho': 1}]

In [26]:
termos_documento= list(map(len, corpus_sem_stopwors_tokenizada))

In [27]:
termos_documento

[8, 9]

In [28]:
result={}
def calculaFT(ocorrencias_termo, termos_documento):
    for i in range(len(ocorrencias_termo)):
        result[i]= dict(map(lambda kv: (kv, ocorrencias_termo[i][kv]/termos_documento[i]), ocorrencias_termo[i]))
    return result

In [29]:
frequencia_termo= calculaFT(ocorrencias_termo, termos_documento)

In [30]:
import pandas as pd
frequencia_termo_df = pd.concat({k: pd.Series(v) for k, v in frequencia_termo.items()}).reset_index()
frequencia_termo_df.columns = ['corpus', 'palavra', 'TF ']

In [31]:
frequencia_termo_df

,corpus,palavra,TF
0,0,rápida,0.125000
1,0,raposa,0.250000
2,0,marrom,0.125000
3,0,pula,0.125000
4,0,sobr,0.125000
5,0,cão,0.125000
6,0,preguiçoso,0.125000
7,1,rápida,0.111111
8,1,raposa,0.222222
9,1,marrom,0.111111


###  Teste com o sciki-learning

In [32]:
corpus_sem_stopwors_tokenizada_list = []
for i in range (len(corpus)):
    corpus_sem_stopwors_tokenizada_list.append(convert_list_to_string(corpus_sem_stopwors_tokenizada[i]))

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

vetores = vectorizer.fit_transform(corpus_sem_stopwors_tokenizada_list)
vocab = vectorizer.get_feature_names()

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(vetores.toarray())

Vocabulário
['baralho', 'cão', 'marrom', 'preguiçoso', 'pula', 'rapasa', 'raposa', 'rápida', 'sobr']

Matrix
[[0 1 1 1 1 0 2 1 1]
 [1 0 1 1 1 1 2 1 1]]


In [34]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np

vectorizer = Pipeline([('count', CountVectorizer()),('tfid', TfidfTransformer(use_idf=False))])

#vectorizer = Pipeline([('count', CountVectorizer()),('tfid', TfidfTransformer())])

vetores = vectorizer.fit_transform(corpus_sem_stopwors_tokenizada_list)
vocab = vectorizer['count'].get_feature_names_out()

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(np.round(vetores.toarray(), 2))

Vocabulário
['baralho' 'cão' 'marrom' 'preguiçoso' 'pula' 'rapasa' 'raposa' 'rápida'
 'sobr']

Matrix
[[0.   0.32 0.32 0.32 0.32 0.   0.63 0.32 0.32]
 [0.3  0.   0.3  0.3  0.3  0.3  0.6  0.3  0.3 ]]


In [35]:
import string



vectorizer = Pipeline([('count', CountVectorizer(analyzer='word')),
                 ('tfid', TfidfTransformer())])

vetores = vectorizer.fit_transform(corpus_sem_stopwors_tokenizada_list)
vocab = vectorizer['count'].get_feature_names()

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(np.round(vetores.toarray(), 2))

Vocabulário
['baralho', 'cão', 'marrom', 'preguiçoso', 'pula', 'rapasa', 'raposa', 'rápida', 'sobr']

Matrix
[[0.   0.42 0.3  0.3  0.3  0.   0.6  0.3  0.3 ]
 [0.39 0.   0.28 0.28 0.28 0.39 0.56 0.28 0.28]]


In [36]:
import string

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus_sem_stopwors_tokenizada_list)
vectorizer.get_feature_names_out()

array(['baralho', 'cão', 'marrom', 'preguiçoso', 'pula', 'rapasa',
       'raposa', 'rápida', 'sobr'], dtype=object)

In [37]:
print(X.toarray())

[[0 1 1 1 1 0 2 1 1]
 [1 0 1 1 1 1 2 1 1]]


In [38]:
vectorizer.vocabulary_

{'rápida': 7,
 'raposa': 6,
 'marrom': 2,
 'pula': 4,
 'sobr': 8,
 'cão': 1,
 'preguiçoso': 3,
 'rapasa': 5,
 'baralho': 0}

In [39]:
cv = CountVectorizer()
cv_fit=cv.fit_transform(corpus_sem_stopwors_tokenizada_list)

print(cv.get_feature_names())
print(cv_fit.toarray())

['baralho', 'cão', 'marrom', 'preguiçoso', 'pula', 'rapasa', 'raposa', 'rápida', 'sobr']
[[0 1 1 1 1 0 2 1 1]
 [1 0 1 1 1 1 2 1 1]]


In [40]:
print(cv_fit.toarray().sum(axis=0))

[1 1 2 2 2 1 4 2 2]


In [41]:
word_list = cv.get_feature_names()
word_list

['baralho',
 'cão',
 'marrom',
 'preguiçoso',
 'pula',
 'rapasa',
 'raposa',
 'rápida',
 'sobr']

In [42]:
count_list = np.asarray(X.sum(axis=0)).ravel()
count_list

array([1, 1, 2, 2, 2, 1, 4, 2, 2])

In [43]:
print(dict(zip(word_list, count_list)))

{'baralho': 1, 'cão': 1, 'marrom': 2, 'preguiçoso': 2, 'pula': 2, 'rapasa': 1, 'raposa': 4, 'rápida': 2, 'sobr': 2}


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix
import pandas as pd
import string

#tf = TfidfVectorizer(analyzer='word', stop_words=stopwords, min_df = 0, sublinear_tf=True, use_idf=True)
tf = TfidfVectorizer(analyzer='word', stop_words=stopwords, min_df = 0, sublinear_tf=True, use_idf=True)
#tf = TfidfVectorizer(analyzer='word', stop_words=stopwords, min_df = 0)
tfidf_matrix =  tf.fit_transform(corpus_sem_stopwors_tokenizada_list)
df = pd.DataFrame(tfidf_matrix.toarray(), columns = tf.get_feature_names_out())
print(df)

    baralho       cão    marrom  preguiçoso      pula    rapasa    raposa  \
0  0.000000  0.447999  0.318755    0.318755  0.318755  0.000000  0.539699   
1  0.408845  0.000000  0.290897    0.290897  0.290897  0.408845  0.492531   

     rápida      sobr  
0  0.318755  0.318755  
1  0.290897  0.290897  


In [45]:
feature_names = tf.get_feature_names_out()
print(feature_names)

['baralho' 'cão' 'marrom' 'preguiçoso' 'pula' 'rapasa' 'raposa' 'rápida'
 'sobr']


https://medium.com/analytics-vidhya/demonstrating-calculation-of-tf-idf-from-sklearn-4f9526e7e78b

# 5.2) Document Frequency (DF)
𝐷𝐹 = 𝑞𝑡𝑑 𝑑𝑒 𝑜𝑐𝑜𝑟𝑟ê𝑛𝑐𝑖𝑎 𝑑𝑜 𝑡𝑒𝑟𝑚𝑜 𝑒𝑚 𝑢𝑚 𝑐𝑜𝑛𝑗𝑢𝑛𝑡𝑜 𝑑𝑒 𝑑𝑜𝑐𝑢𝑚𝑒𝑛𝑡𝑜𝑠    
https://kavita-ganesan.com/what-is-document-frequency/#.YagKOCVv894

In [46]:
todas_palavras= []
for i in range (len(ocorrencias_termo)):
    for item in list(ocorrencias_termo[i].keys()):
        if item not in todas_palavras:
            todas_palavras.append(item)

In [47]:
numero_palavras_d1 = dict.fromkeys(todas_palavras, 0)
for word in corpus_sem_stopwors_tokenizada[0]:
        numero_palavras_d1[word] += 1

In [48]:
numero_palavras_d1

{'rápida': 1,
 'raposa': 2,
 'marrom': 1,
 'pula': 1,
 'sobr': 1,
 'cão': 1,
 'preguiçoso': 1,
 'rapasa': 0,
 'baralho': 0}

In [49]:
numero_palavras_d2 = dict.fromkeys(todas_palavras, 0)
for word in corpus_sem_stopwors_tokenizada[1]:
    numero_palavras_d2[word] += 1

In [50]:
documentos = [numero_palavras_d1, numero_palavras_d2]
document_frequency = dict.fromkeys(numero_palavras_d1.keys(), 0)
for document in documentos:
    for word, val in document.items():
        if val > 0:
            document_frequency[word] += 1

In [51]:
documentos

[{'rápida': 1,
  'raposa': 2,
  'marrom': 1,
  'pula': 1,
  'sobr': 1,
  'cão': 1,
  'preguiçoso': 1,
  'rapasa': 0,
  'baralho': 0},
 {'rápida': 1,
  'raposa': 2,
  'marrom': 1,
  'pula': 1,
  'sobr': 1,
  'cão': 0,
  'preguiçoso': 1,
  'rapasa': 1,
  'baralho': 1}]

In [52]:
document_frequency

{'rápida': 2,
 'raposa': 2,
 'marrom': 2,
 'pula': 2,
 'sobr': 2,
 'cão': 1,
 'preguiçoso': 2,
 'rapasa': 1,
 'baralho': 1}

In [53]:
pd.DataFrame.from_dict(document_frequency, orient='index', columns=['DF'])

,DF
rápida,2
raposa,2
marrom,2
pula,2
sobr,2
cão,1
preguiçoso,2
rapasa,1
baralho,1


# 5.3) Inverse Document Frequency (IDF)
𝐼𝐷𝐹 = 𝑙𝑜𝑔(𝑞𝑡𝑑 𝑑𝑒 𝑑𝑜𝑐𝑢𝑚𝑒𝑛𝑡𝑜𝑠 / (𝐷𝐹 + 1))

Referências:           
[stackoverflow](https://stackoverflow.com/questions/48431173/is-there-a-way-to-get-only-the-idf-values-of-words-using-scikit-or-any-other-pyt)     
[scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer


tf = TfidfVectorizer(use_idf=True)
tf.fit_transform(corpus)

idf = tf.idf_ 

In [55]:
idf

array([1.40546511, 1.40546511, 1.40546511, 1.        , 1.        ,
       1.        , 1.        , 1.40546511, 1.        , 1.        ,
       1.        , 1.        ])

In [56]:
idf[tf.vocabulary_["cão"]]

1.4054651081081644

In [57]:
corpus_sem_stopwors_tokenizada= list(map(convert_list_to_string, 
                                         corpus_sem_stopwors_tokenizada))

In [58]:
corpus_sem_stopwors_tokenizada

['rápida raposa marrom pula sobr cão preguiçoso raposa',
 'rápida raposa marrom pula sobr raposa preguiçoso rapasa baralho']

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer


tf = TfidfVectorizer(use_idf=True)
tf.fit_transform(corpus_sem_stopwors_tokenizada)

idf = tf.idf_ 

In [60]:
print(idf.shape)

(9,)


In [61]:
idf

array([1.40546511, 1.40546511, 1.        , 1.        , 1.        ,
       1.40546511, 1.        , 1.        , 1.        ])

In [62]:
idf[tf.vocabulary_["cão"]]

1.4054651081081644

# 5.4) TF-IDF
𝑇𝐹 − 𝐼𝐷𝐹 = 𝐼𝐷𝐹 * 𝑇𝐹

In [63]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline


nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

#######
# https://stackoverflow.com/questions/26126442/combining-text-stemming-and-removal-of-punctuation-in-nltk-and-scikit-learn
# based on http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(texto):
    tokens = nltk.word_tokenize(texto)
    tokens = [i for i in tokens if i not in string.punctuation]
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer = Pipeline([('count', CountVectorizer(analyzer='word', stop_words=stopwords)),
                 ('tfid', TfidfTransformer(use_idf=False))])

vetores = vectorizer.fit_transform(corpus)
vocab = vectorizer['count'].get_feature_names_out()

print('Vocabulário')
print(vocab)
print()
print('Matrix')
print(np.round(vetores.toarray(), 2))

Vocabulário
['baralho' 'cão' 'marrom' 'preguiçoso' 'pula' 'rapasa' 'raposa' 'rápida'
 'sobre']

Matrix
[[0.   0.32 0.32 0.32 0.32 0.   0.63 0.32 0.32]
 [0.3  0.   0.3  0.3  0.3  0.3  0.6  0.3  0.3 ]]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vagnersv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 5.5) Lista de strings com proximidade até 2 dos 5 termos de maior TF-IDF. Essas strings devem ser acompanhadas de seu valor de TF.

# 6) Gerar um arquivo csv que possui todas as palavras de todos os documentos na primeira coluna, em que cada linha é um token. Para cada token, informe nas colunas vizinhas as informações determinadas no objetivo 5.

# 7) Gerar nuvem de palavras para análise visual tal como exemplo abaixo. Cada ponto central será um dos 5 termos de maior TF-IDF. As conexões são as palavras próximas obtidas em 5.4. O tamanho do círculo da palavra é baseado no TF dela. O maior círculo que conecta o termo central será normalizado para palavras de maior TF do conjunto (desafio)

# Tópicos de Auxílio
Se realizada a lematização manual, os resultados seguintes são duplicados e a comparação será realizada analisando a nuvem de palavras de cada implementação.
Informações sobre as métricas utilizadas
https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-worlddataset-
796d339a4089
Atividade desafio de determinação da nuvem de palavras
https://www.kaggle.com/arthurtok/ghastly-network-and-d3-js-force-directed-graphs

http://andrewtrick.com/stormlight_network.html

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

my_data = ["hello how are you", "hello who are you", "i am not you"]

tf = TfidfVectorizer(use_idf=True)
tf.fit_transform(my_data)

idf = tf.idf_ 

#[BONUS] if you want to get the idf value for a particular word:

# If you want to get the idf value for a particular word, here "hello"    
tf.idf_[tf.vocabulary_["hello"]]


1.2876820724517808

In [65]:
len(my_data)

3

In [66]:
my_data[0]

'hello how are you'